# Granite-TimeSeries-TTM   

TinyTimeMixers (TTMs) are compact pre-trained models for Multivariate Time-Series Forecasting, open-sourced by IBM Research. With less than 1 Million parameters, TTM introduces the notion of the first-ever "tiny" pre-trained models for Time-Series Forecasting. TTM outperforms several popular benchmarks demanding billions of parameters in zero-shot and few-shot forecasting and can easily be fine-tuned for multi-variate forecasts.

In [1]:
import pathlib

import pandas as pd

from tsfm_public import TimeSeriesForecastingPipeline, TinyTimeMixerForPrediction
from tsfm_public.toolkit.visualization import plot_predictions

INFO:p-75334:t-8214683904:config.py:<module>:PyTorch version 2.3.0 available.


In [2]:
import tsfm_public


tsfm_public.__version__

'0.2.10.dev14+gc889210.d20240918'

## Initial setup
1. Download energy_data.csv.zip and weather_data.csv.zip from https://www.kaggle.com/datasets/nicholasjhana/energy-consumption-generation-prices-and-weather
2. Place the downloaded files into a folder and update the data_path below

In [5]:
data_path = pathlib.Path("~/Downloads")

## Load and prepare data

In [6]:
# Download energy_data.csv.zip from https://www.kaggle.com/datasets/nicholasjhana/energy-consumption-generation-prices-and-weather

dataset_path = data_path / "energy_dataset.csv.zip"
timestamp_column = "time"

target_column = "total load actual"

context_length = 512  # set by the pretrained model we will use

data = pd.read_csv(
    dataset_path,
    parse_dates=[timestamp_column],
)

data = data.ffill()

data = data.iloc[-context_length:,]

print(data.shape)
data.head()

(512, 29)


time  generation biomass  \
34552  2018-12-10 16:00:00+01:00               308.0   
34553  2018-12-10 17:00:00+01:00               314.0   
34554  2018-12-10 18:00:00+01:00               313.0   
34555  2018-12-10 19:00:00+01:00               315.0   
34556  2018-12-10 20:00:00+01:00               316.0   

       generation fossil brown coal/lignite  \
34552                                 683.0   
34553                                 686.0   
34554                                 711.0   
34555                                 716.0   
34556                                 711.0   

       generation fossil coal-derived gas  generation fossil gas  \
34552                                 0.0                 3978.0   
34553                                 0.0                 4338.0   
34554                                 0.0                 5020.0   
34555                                 0.0                 5449.0   
34556                                 0.0                 5645.0   

       generation fossil hard coal  generation fossil oil  \
34552                       3080.0                  306.0   
34553                       3241.0                  303.0   
34554                       3436.0                  305.0   
34555                       3410.0                  294.0   
34556                       3419.0                  295.0   

       generation fossil oil shale  generation fossil peat  \
34552                          0.0                     0.0   
34553                          0.0                     0.0   
34554                          0.0                     0.0   
34555                          0.0                     0.0   
34556                          0.0                     0.0   

       generation geothermal  ...  generation waste  generation wind offshore  \
34552                    0.0  ...             289.0                       0.0   
34553                    0.0  ...             289.0                       0.0   
34554                    0.0  ...             295.0                       0.0   
34555                    0.0  ...             297.0                       0.0   
34556                    0.0  ...             294.0                       0.0   

       generation wind onshore  forecast solar day ahead  \
34552                   5746.0                    2494.0   
34553                   5524.0                    1838.0   
34554                   5139.0                    1119.0   
34555                   4933.0                     404.0   
34556                   4929.0                     200.0   

       forecast wind offshore eday ahead  forecast wind onshore day ahead  \
34552                                NaN                           6466.0   
34553                                NaN                           6269.0   
34554                                NaN                           5962.0   
34555                                NaN                           5690.0   
34556                                NaN                           5680.0   

       total load forecast  total load actual  price day ahead  price actual  
34552              24484.0            24465.0            56.96         69.76  
34553              24033.0            24068.0            67.32         73.48  
34554              24053.0            24018.0            68.68         77.65  
34555              25203.0            25036.0            70.46         76.23  
34556              27579.0            27411.0            72.82         75.54  

[5 rows x 29 columns]

## Load pretrained Granite-TimeSeries-TTM model (zero-shot)
The **TTM** model supports huggingface model interface, allowing easy API for loading the saved models.

In [7]:
zeroshot_model = TinyTimeMixerForPrediction.from_pretrained(
    "ibm-granite/granite-timeseries-ttm-v1", num_input_channels=1
)
model_parameters = sum(p.numel() for p in zeroshot_model.parameters() if p.requires_grad)
print("TTM Model parameters:", model_parameters)

TTM Model parameters: 805280


### Create a time series forecasting pipeline

In [8]:
pipeline = TimeSeriesForecastingPipeline(
    zeroshot_model,
    timestamp_column=timestamp_column,
    target_columns=[target_column],
    explode_forecasts=True,
    freq="h",
    id_columns=[],
)
zeroshot_forecast = pipeline(data)
zeroshot_forecast.head()

time  total load actual_prediction
0 2019-01-01 00:00:00+01:00                  23504.996094
1 2019-01-01 01:00:00+01:00                  22338.626953
2 2019-01-01 02:00:00+01:00                  21448.902344
3 2019-01-01 03:00:00+01:00                  20982.527344
4 2019-01-01 04:00:00+01:00                  20697.183594

### Plot the results

In [9]:
plot_predictions(
    input_df=data,
    exploded_predictions_df=zeroshot_forecast,
    freq="h",
    timestamp_column=timestamp_column,
    channel=target_column,
    indices=[-1],
    num_plots=1,
)

## Useful links

TinyTimeMixer paper: https://arxiv.org/abs/2401.03955  

Granite-TimeSeries-TTM model: https://huggingface.co/ibm-granite/granite-timeseries-ttm-v1  

Publicly available tools for working with our models: https://github.com/ibm-granite/granite-tsfm

© 2024 IBM Corporation